In [6]:
from huggingface_hub import login
login()

# SFT + DPO

## SFT

In [1]:
from simplet5_trl import SimpleT5_TRL
import pandas as pd

In [2]:
train_df = pd.read_csv("data/sft_train.csv")
train_df.head(2)

,source_text,target_text
0,summarize: Artificial intelligence is revoluti...,AI transforms healthcare through faster diagno...
1,summarize: Climate change poses significant ch...,Climate change threatens food security through...


In [3]:
val_df = pd.read_csv("data/sft_val.csv")
val_df.head(2)

,source_text,target_text
0,summarize: Robotics automation is transforming...,Robotics transforms manufacturing with collabo...
1,summarize: Cloud computing has become essentia...,Cloud computing provides essential scalable in...


In [4]:
# Initialize and load model
model = SimpleT5_TRL()
model.from_pretrained("facebook/bart-base")

Loading weights:   0%|          | 0/259 [00:00<?, ?it/s]

In [5]:
model.train(
    train_df=train_df,
    eval_df=val_df,
    source_max_token_len=512,
    target_max_token_len=128,
    batch_size=4,
    max_epochs=3,
    learning_rate=1e-4,
    outputdir="outputs/longt5_sft",
    save_strategy="epoch",
    save_total_limit=2,
    precision="32"
)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

2025-12-30 20:31:42.280271: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.796371,2.421082,40.915700,25.034300,40.915700,40.915700
2,1.584836,2.383382,39.461600,17.989200,38.767200,38.767200
3,1.673006,2.337334,44.924100,21.848900,42.979700,42.979700


In [6]:
model.load_model("outputs/longt5_sft/checkpoint-45", use_gpu=True)
print(model.predict("translate English to French: Good Morning!"))

Loading weights:   0%|          | 0/260 [00:00<?, ?it/s]

['Le progres est le recherche.']


## DPO

In [7]:
train_df = pd.read_csv("data/dpo_train.csv")
eval_df = pd.read_csv("data/dpo_val.csv")

In [8]:
model.train_dpo(
    train_df=train_df,
    eval_df=eval_df,
    beta=0.1,                    # Lower = more deviation from reference allowed
    loss_type="sigmoid",         # Standard DPO loss
    max_length=512,
    max_prompt_length=256,
    batch_size=4,
    max_epochs=3,
    learning_rate=5e-7,          # DPO typically uses lower learning rates
    outputdir="outputs/dpo",
)

Extracting prompt in train dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
1,0.979697,0.598648,0.076292,-0.124239,1.000000,0.200532,-76.404793,-70.159073,-2.303445,-1.402496
2,1.680983,0.552758,0.113599,-0.193935,1.000000,0.307534,-76.031715,-70.856026,-2.296139,-1.392909
3,1.237100,0.536807,0.128044,-0.218316,1.000000,0.346360,-75.887268,-71.099831,-2.295274,-1.391303


In [9]:
model.load_model("outputs/dpo/checkpoint-60", use_gpu=True)
print(model.predict("translate English to French: Good Morning!"))

Loading weights:   0%|          | 0/260 [00:00<?, ?it/s]

['Le croissance est la progresse.']


# SFT + SimPO

In [10]:
from simplet5_trl import SimpleT5_TRL
import pandas as pd

In [11]:
train_df = pd.read_csv("data/sft_train.csv")
val_df = pd.read_csv("data/sft_val.csv")

In [12]:
# Initialize and load model
model = SimpleT5_TRL()
model.from_pretrained("google/t5gemma-s-s-ul2")

Loading weights:   0%|          | 0/228 [00:00<?, ?it/s]

In [13]:
model.train(
    train_df=train_df,
    eval_df=val_df,
    source_max_token_len=512,
    target_max_token_len=128,
    batch_size=4,
    max_epochs=5,
    learning_rate=1e-4,
    outputdir="outputs/t5gemma_sft",
    save_strategy="epoch",
    save_total_limit=2,
)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,9.955886,9.001279,8.764900,0.000000,8.764900,8.764900
2,7.426774,8.053118,4.821400,0.000000,4.821400,4.821400
3,5.819927,7.520780,6.058700,0.000000,6.058700,6.058700
4,3.373345,7.925241,2.248500,0.000000,2.248500,2.248500
5,2.783420,8.417527,3.140800,0.347200,2.811800,2.811800


In [14]:
model.load_model("outputs/t5gemma_sft/checkpoint-75", use_gpu=True)
print(model.predict("translate English to French: Good Morning!"))

Loading weights:   0%|          | 0/228 [00:00<?, ?it/s]

["L'innovation stimule le succes.ici vendredi.on jamais.est est la fort des opportunites.on.on.on.on. intensification.on.on.ens. innovation. training. crops. enhances battery. fusion. plusite. and retail. collaboration. organizational agility. productivity. cle. companies. genomic.. reuse..pes work. computers. reducing emissions. improving personalized.. manufacturing. services. treatments. communication. global. targeted.. investments.. connectivity... ecosystems...... applications...................... farming........................................................................................................................................................................................................................................................................................................................................................................................"]


## SimPO

In [15]:
train_df = pd.read_csv("data/simpo_train.csv")
eval_df = pd.read_csv("data/simpo_val.csv")

In [16]:
model.train_simpo(
    train_df=train_df,
    eval_df=eval_df,
    beta=2.0,                    # SimPO uses higher beta than DPO
    simpo_gamma=0.5,             # Target reward margin
    label_smoothing=0.0,
    max_length=512,
    max_prompt_length=256,
    batch_size=4,
    max_epochs=3,
    learning_rate=5e-7,
    finetuning="lora",
    outputdir="outputs/simpo",
)

trainable params: 1,572,864 || all params: 314,090,496 || trainable%: 0.5008


<string>:129: FutureWarning: The `CPOConfig` is now located in `trl.experimental`. Please update your imports to `from trl.experimental.cpo import CPOConfig`. The current import path will be removed and no longer supported in TRL 0.29. For more information, see https://github.com/huggingface/trl/issues/4223.
/home/sagemaker-user/simpleT5-TRL/simplet5_trl/simplet5_trl.py:1898: FutureWarning: The `CPOTrainer` is now located in `trl.experimental`. Please update your imports to `from trl.experimental.cpo import CPOTrainer`. The current import path will be removed and no longer supported in TRL 0.29. For more information, see https://github.com/huggingface/trl/issues/4223.
  self.trainer = Seq2SeqCPOTrainerCompat(


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss
1,7.531250,9.118750,0.302700,66.067000,16.517000,-17.375000,-26.150000,0.950000,8.743750,-13.075000,-8.687500,7.256250,-1.633887,8.818750
2,8.562500,9.112500,0.303400,65.930000,16.482000,-17.350000,-26.125000,0.950000,8.768750,-13.062500,-8.675000,7.262500,-1.622852,8.812500
3,9.187500,9.106250,0.305600,65.447000,16.362000,-17.350000,-26.125000,0.950000,8.762500,-13.062500,-8.675000,7.262500,-1.616943,8.806250


In [17]:
print(model.predict("translate English to French: Good Morning!"))

["L'innovation stimule le succes.onon jamais.est en une opportunites.ici vendredi. intensification. agility.arrete jamais. innovation. crops. advances. enhances access. organizational agility. productivity. connectivity. genomic. fusion. manufacturing and healthcare. reuse. collaboration. computers. services. adoption. improving battery. training. companies. reducing pollution. personalized. treatments. farming. faster diagnosis. communication. industry. processing. global. applications. prioriti. targeted.. investments. ecosystems. cities....... plus for solutions...................... exploration........................................................................................................................................................................................................................................................................................................................................................................................"]


# SFT + RFT

In [18]:
from simplet5_trl import SimpleT5_TRL
import pandas as pd

In [19]:
train_df = pd.read_csv("data/sft_train.csv")
val_df = pd.read_csv("data/sft_val.csv")

In [20]:
# Initialize and load model
model = SimpleT5_TRL()
model.from_pretrained("google-t5/t5-small")

Loading weights:   0%|          | 0/131 [00:00<?, ?it/s]

In [21]:
model.train(
    train_df=train_df,
    eval_df=val_df,
    source_max_token_len=512,
    target_max_token_len=128,
    batch_size=4,
    max_epochs=5,
    learning_rate=1e-4,
    outputdir="outputs/t5_sft",
    save_strategy="epoch",
    save_total_limit=2,
)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.509808,1.744200,62.848800,39.147500,62.848800,62.848800
2,2.161224,1.642760,66.050400,41.372200,66.050400,66.050400
3,1.642640,1.600037,70.265800,46.005900,70.265800,70.265800
4,1.507864,1.583732,70.265800,46.005900,70.265800,70.265800
5,1.840633,1.574657,70.265800,46.005900,70.265800,70.265800


In [22]:
model.load_model("outputs/t5_sft/checkpoint-75", use_gpu=True)
print(model.predict("translate English to French: Good Morning!"))

Loading weights:   0%|          | 0/131 [00:00<?, ?it/s]

['Bonjour!']


## RFT

In [23]:
train_df = pd.read_csv("data/rft_train.csv")
eval_df = pd.read_csv("data/rft_val.csv")

In [24]:
model.train_rft(
    train_df=train_df,
    eval_df=eval_df,
    max_seq_length=512,
    batch_size=8,
    max_epochs=3,
    learning_rate=2e-5,
    finetuning="lora",
    lora_r=16,
    outputdir="outputs/rft",
)

trainable params: 1,179,648 || all params: 61,686,272 || trainable%: 1.9123


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.789991,3.317001,40.023200,23.070100,37.013400,37.013400
2,3.623442,3.312031,40.454600,22.967600,37.444800,37.444800
3,2.651294,3.310171,40.454600,22.967600,37.444800,37.444800


In [25]:
model.load_model("outputs/rft/checkpoint-30", use_gpu=True)
print(model.predict("translate English to French: Good Morning! How are you?"))

Loading weights:   0%|          | 0/131 [00:00<?, ?it/s]

["Le matin, c'est-à-dire que vous êtes bien?"]


# BART/PEGASUS/T5GEMMA2 testing

In [1]:
from simplet5_trl import SimpleT5_TRL
import pandas as pd

In [2]:
def test_model(model_name):
    train_df = pd.read_csv("data/sft_train.csv")
    val_df = pd.read_csv("data/sft_val.csv")
    # Initialize and load model
    model =  SimpleT5_TRL()
    model.from_pretrained(model_name)
    model.train(
    train_df=train_df,
    eval_df=val_df,
    source_max_token_len=512,
    target_max_token_len=128,
    batch_size=4,
    max_epochs=5,
    learning_rate=1e-4,
    outputdir="outputs/t5_sft",
    save_strategy="epoch",
    save_total_limit=2,
    )
    train_df = pd.read_csv("data/rft_train.csv")
    eval_df = pd.read_csv("data/rft_val.csv")
    model.train_rft(
    train_df=train_df,
    eval_df=eval_df,
    max_seq_length=128,
    batch_size=8,
    max_epochs=3,
    learning_rate=2e-5,
    finetuning="lora",
    lora_r=8,
    outputdir="outputs/rft",
    )
    model.load_model("outputs/rft/checkpoint-30", use_gpu=True)
    print(model.predict("translate English to French: Good Morning! How are you?"))

In [3]:
test_model("facebook/bart-base")

Loading weights:   0%|          | 0/259 [00:00<?, ?it/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

2025-12-30 20:44:40.158861: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.806641,2.452336,40.314700,25.034300,40.314700,40.314700
2,1.650561,2.526084,38.646900,20.200400,37.952500,37.952500
3,1.486393,2.399581,42.512700,23.264700,40.735700,40.735700
4,1.307119,2.343822,36.913300,17.348700,34.928000,34.928000
5,0.434856,2.361337,39.248700,18.621200,36.221700,36.221700


...

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,4.438905,4.486356,26.449400,10.139800,22.973800,22.973800
2,3.953916,4.443602,25.625900,8.473200,21.562000,21.562000
3,4.737692,4.428769,25.625900,8.473200,21.562000,21.562000


Loading weights:   0%|          | 0/259 [00:00<?, ?it/s]

['translate English to French: Good Morning! How are you?']


In [4]:
test_model("google/pegasus-xsum")

Loading weights:   0%|          | 0/678 [00:00<?, ?it/s]

PegasusForConditionalGeneration LOAD REPORT from: google/pegasus-xsum
Key                                  | Status  | 
-------------------------------------+---------+-
model.decoder.embed_positions.weight | MISSING | 
model.encoder.embed_positions.weight | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


...

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.377697,3.276729,25.339900,9.148200,24.126700,24.126700
2,3.379344,3.271930,25.339900,9.148200,24.126700,24.126700
3,3.517129,3.270120,25.339900,9.148200,24.126700,24.126700


Loading weights:   0%|          | 0/678 [00:00<?, ?it/s]

PegasusForConditionalGeneration LOAD REPORT from: google/pegasus-xsum
Key                                  | Status  | 
-------------------------------------+---------+-
model.decoder.embed_positions.weight | MISSING | 
model.encoder.embed_positions.weight | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


['How are you?']


In [3]:
test_model("google/t5gemma-2-270m-270m")

Loading weights:   0%|          | 0/911 [00:00<?, ?it/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

2025-12-30 20:49:50.945706: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,4.186065,3.351251,7.524900,1.986600,6.546700,6.546700
2,1.921416,3.737850,6.850500,1.223900,6.242700,6.242700
3,0.804530,4.083356,6.003900,1.696300,5.642000,5.642000
4,0.361469,3.786724,5.305100,1.628200,4.967800,4.967800
5,0.115626,3.918174,5.696100,1.378500,5.196600,5.196600


...

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,5.854585,6.060541,4.165600,1.217400,4.040600,4.040600
2,5.160693,5.713160,5.990500,1.447700,5.063900,5.063900
3,5.907971,5.591557,3.604200,1.218800,3.418800,3.418800


Loading weights:   0%|          | 0/911 [00:00<?, ?it/s]

[" Are you feeling better? What are you doing today? How are you? I hope you feel better. Have a good day! If you have any questions or comments, please don't hesitate to contact me at 610-238-4759. Thank you for your business and look forward to hearing from you soon."]
